<a href="https://colab.research.google.com/github/AbhinavJain3/Purchase-Order-Assignment--Abhinav-Jain/blob/main/PurchaseOrderAssignment_Abhinav_Jain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx
!pip install pandas
!pip install PyPDF2
!pip install imapclient
!pip install -q llama-index-llms-azure-openai
!pip install -q llama-index-embeddings-azure-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from IPython import get_ipython
import os
import imaplib
import email
import base64
import PyPDF2
import re
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core.llms import ChatMessage
import json
import pandas as pd
from IPython.display import display, HTML

In [ ]:
def setup_llm():
    os.environ["OPENAI_API_KEY"] = "your_api_here"
    os.environ["AZURE_OPENAI_ENDPOINT"] = "your_openai_endpoint"
    os.environ["OPENAI_API_VERSION"] = "your_api_version"
    return AzureOpenAI(
        engine="gpt-4o",
        model="gpt-4o",
        temperature=0.0,
        azure_endpoint="your_openai_endpoint",
        api_key="your_api_here",
        api_version="your_api_version",
    )

In [ ]:
llm = setup_llm()

In [ ]:
def fetch_recent_emails(mail, limit=10, download_path='attachments'):
    os.makedirs(download_path, exist_ok=True)
    status, messages = mail.search(None, 'ALL')

    email_list = []
    for num in messages[0].split()[-limit:]:
        status, msg_data = mail.fetch(num, '(RFC822)')
        raw_email = msg_data[0][1]
        email_message = email.message_from_bytes(raw_email)

        email_data = {
            'subject': email_message['Subject'],
            'from': email_message['From'],
            'body': _extract_email_body(email_message),
            'attachments': []
        }

        for part in email_message.walk():
            if part.get_content_maintype() == 'multipart':
                continue

            filename = part.get_filename()
            if filename:
                filepath = os.path.join(download_path, filename)

                try:
                    # More flexible payload extraction
                    payload = part.get_payload(decode=True)
                    if payload:
                        with open(filepath, 'wb') as f:
                            f.write(payload)

                        email_data['attachments'].append({
                            'filename': filename,
                            'filepath': filepath,
                            'type': part.get_content_type()
                        })
                except Exception as e:
                    print(f"Detailed Error: {e}")
                    print(f"Part Details: {part.keys()}")

        email_list.append(email_data)

    return email_list

def _extract_email_body(email_message):
    body = ""
    if email_message.is_multipart():
        for part in email_message.walk():
            if part.get_content_type() == "text/plain":
                body = part.get_payload(decode=True).decode()
                break
    else:
        body = email_message.get_payload(decode=True).decode()

    return body

def debug_email_parts(email_message):
    for part in email_message.walk():
        print("Part Content Type:", part.get_content_type())
        print("Part Headers:", part.keys())
        print("Filename:", part.get_filename())
        print("Encoding:", part.get('Content-Transfer-Encoding'))
        print("---")

# Usage remains the same
def connect_to_email(email_address, password, imap_server='imap.gmail.com'):
    try:
        mail = imaplib.IMAP4_SSL(imap_server)
        mail.login(email_address, password)
        mail.select('inbox')
        return mail
    except Exception as e:
        print(f"Connection error: {e}")
        return None

In [ ]:
mail = connect_to_email('jethalalg426@gmail.com', '**** **** **** ****')
if mail:
    emails_with_attachments = fetch_recent_emails(mail, limit=5)

    # Inspect emails and their attachments
    for email in emails_with_attachments:
        print("Subject:", email['subject'])
        print("Attachments:", email['attachments'])

Subject: Security alert
Attachments: []
Subject: Purchase Order
Attachments: [{'filename': 'PurchaseOrder (1).pdf', 'filepath': 'attachments/PurchaseOrder (1).pdf', 'type': 'application/pdf'}]
Subject: =?UTF-8?B?8J+Tjg==?= Attached: Your Loan Offer!
 =?UTF-8?B?4oK5MTUsMDAsMDAw?= waiting for approval!
Attachments: []
Subject: Start your free trial to The Economist
Attachments: []
Subject: "Forbidden names "
Attachments: []


In [ ]:
def classify_purchase_orders(emails):
    """
    Advanced email classification for purchase orders with confidence scoring.

    Args:
        emails: List of email dictionaries with 'subject', 'body', 'from' keys.

    Returns:
        List of dictionaries with purchase order emails and their confidence scores.
    """
    purchase_orders = []

    # Expanded keyword lists for more robust detection
    po_subject_keywords = [
        r'purchase\s*order', r'po', r'order\s*confirmation',
        r'procurement', r'buying\s*request', r'product\s*order'
    ]

    po_body_keywords = [
        r'quantity', r'unit\s*price', r'total\s*amount',
        r'delivery\s*date', r'billing\s*details', r'purchase\s*details'
    ]

    for email_data in emails:
        subject = email_data.get('subject', '').lower()
        body = email_data.get('body', '').lower()
        sender = email_data.get('from', '').lower()

        confidence_score = 0

        # Subject-based scoring
        subject_matches = sum(bool(re.search(kw, subject, re.IGNORECASE)) for kw in po_subject_keywords)
        confidence_score += subject_matches * 30  # Higher weight for subject matches

        # Body-based scoring
        body_matches = sum(bool(re.search(kw, body, re.IGNORECASE)) for kw in po_body_keywords)
        confidence_score += body_matches * 20

        # Sender domain scoring (optional enhancement)
        business_domains = ['procurement', 'purchase', 'sales', 'order', 'vendor']
        sender_domain_match = any(domain in sender for domain in business_domains)
        if sender_domain_match:
            confidence_score += 10

        # Threshold for classification
        if confidence_score >= 30:  # Adjustable threshold
            purchase_orders.append({
                'email': email_data,
                'confidence_score': confidence_score
            })

    return sorted(purchase_orders, key=lambda x: x['confidence_score'], reverse=True)

In [ ]:
classified_pos = classify_purchase_orders(emails_with_attachments)
for po in classified_pos:
    #print(f"Confidence: {po['confidence_score']}%")
    # Further processing
    print(po)

{'email': {'subject': 'Purchase Order', 'from': 'Abhinav Jain <abhinavj165@gmail.com>', 'body': 'Dear Mam,\r\n\r\nPlease find attached herewith the purchase order.\r\n\r\nRegards,\r\nAbhinav Jain\r\n', 'attachments': [{'filename': 'PurchaseOrder (1).pdf', 'filepath': 'attachments/PurchaseOrder (1).pdf', 'type': 'application/pdf'}]}, 'confidence_score': 30}


In [ ]:
def extract_pdf_text(filepath):
    with open(filepath, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def process_purchase_order_pdf(pdf_filepath):
    # Extract text from PDF
    pdf_text = extract_pdf_text(pdf_filepath)

    # Define key fields to extract (customize based on your PO format)
    key_fields = {
        'po_number': r'Purchase Order Number:\s*(\w+)',
        'customer_name': r'Customer Name:\s*(.+)',
        'total_quantity': r'Total Quantity:\s*(\d+)',
        'total_amount': r'Total Amount:\s*\$?(\d+\.\d{2})'
    }

    # Extract fields using regex
    extracted_data = {}
    for field, pattern in key_fields.items():
        match = re.search(pattern, pdf_text, re.IGNORECASE)
        if match:
            extracted_data[field] = match.group(1)

    return {
        'full_text': pdf_text,
        'extracted_fields': extracted_data
    }



In [ ]:
# Usage in email processing
for email in emails_with_attachments:
    for attachment in email['attachments']:
        if attachment['type'] == 'application/pdf':
            pdf_data = process_purchase_order_pdf(attachment['filepath'])
            print(pdf_data)

In [ ]:
def clean_json_blocks(text):
    return text.replace("```json", "").replace("```", "").strip()

In [ ]:
def extract_po_details(llm, input_data):
    """
    Extract structured information from the provided input data using the LLM.

    Parameters:
        llm: The language model instance used for processing.
        input_data: Dictionary containing the `full_text` field with raw input data.

    Returns:
        A dictionary with extracted and structured details.
    """
    # Define the structure and expectations
    structure = """
    Extract and structure the following details:
    - Customer PO Number (mandatory)
    - Item Name (mandatory)
    - Quantity (mandatory)
    - Rate per unit (mandatory)
    - Unit of measurement (mandatory)
    - Item wise Delivery Dates (mandatory)
    - Customer Name (mandatory)
    - Customer details (like phone number, email, GST number) (Optional)
    - Applicable Taxes (Optional)
    - Terms of Payment (Optional)
    - Discount (Optional)
    - Other remarks/instructions (Optional)

    Provide the output in JSON format.
    """

    # Prepare the input for the LLM
    messages = [
        ChatMessage(
            role="system",
            content=f"Extract the following structured details: {structure}."
        ),
        ChatMessage(
            role="user",
            content=f"Input data: {input_data['full_text']}"
        )
    ]

    # Use the LLM to extract structured details
    response = llm.chat(messages).message.content
    resp = clean_json_blocks(response)
    print(response)

    return resp

In [ ]:
ab = extract_po_details(llm, pdf_data)

```json
{
  "Customer PO Number": "24250341",
  "Customer Name": "Vaishnav Fastners",
  "Customer Details": {
    "Address": "Vaishnav Fastners Gala No.25, Blue Rose Ind. Est, Opp Magathane Bus Depot, Borivali(E), Mumbai",
    "GST Number": "27AAHPS2256F1ZM"
  },
  "Items": [
    {
      "Item Name": "Travalent screw washer",
      "Quantity": 300000,
      "Rate per unit": 0.98,
      "Unit of measurement": "Nos",
      "Delivery Dates": {
        "04/12": 60000,
        "08/12": 60000,
        "12/12": 60000,
        "18/12": 60000,
        "24/12": 60000
      }
    },
    {
      "Item Name": "SCREW WASHER ASSEMBLY MO-0",
      "Quantity": 160000,
      "Rate per unit": 0.97,
      "Unit of measurement": "Nos",
      "Delivery Dates": {
        "04/12": 20000,
        "10/12": 20000,
        "20/12": 40000,
        "25/12": 40000,
        "30/12": 40000
      }
    }
  ],
  "Applicable Taxes": {
    "CGST": "9%",
    "SGST": "9%",
    "IGST": "0%",
    "UTGST": "0%"
  },
  "Terms o

In [ ]:
def format_purchase_order(po_data):
    """
    Format and display purchase order details in a readable manner

    Args:
        po_data (dict): Purchase order dictionary
    """
    # Create HTML for styling
    html_content = f"""
    <style>
        .po-container {{
            font-family: Arial, sans-serif;
            max-width: 1000px;
            margin: 20px auto;
            padding: 20px;
            border: 1px solid #ddd;
            border-radius: 8px;
        }}
        .po-section {{
            margin-bottom: 20px;
            padding: 15px;
            background-color: #f9f9f9;
            border-radius: 5px;
        }}
        .po-title {{
            font-size: 18px;
            font-weight: bold;
            color: #333;
            margin-bottom: 10px;
            border-bottom: 2px solid #ddd;
            padding-bottom: 5px;
        }}
    </style>
    <div class="po-container">
        <div class="po-section">
            <div class="po-title">Customer Information</div>
            <p><strong>PO Number:</strong> {po_data['Customer PO Number']}</p>
            <p><strong>Customer Name:</strong> {po_data['Customer Name']}</p>
            <p><strong>Address:</strong> {po_data['Customer Details']['Address']}</p>
            <p><strong>GST Number:</strong> {po_data['Customer Details']['GST Number']}</p>
        </div>

        <div class="po-section">
            <div class="po-title">Items</div>
    """

    # Items table
    items_data = []
    for item in po_data['Items']:
        delivery_dates = ', '.join([f"{date}: {qty} units" for date, qty in item['Delivery Dates'].items()])
        items_data.append({
            'Item Name': item['Item Name'],
            'Quantity': f"{item['Quantity']:,}",
            'Rate per Unit': f"₹{item['Rate per unit']:.2f}",
            'Unit': item['Unit of measurement'],
            'Delivery Schedule': delivery_dates
        })

    items_df = pd.DataFrame(items_data)
    html_content += items_df.to_html(index=False, classes='table table-striped')

    # Taxes and Payment
    html_content += f"""
        </div>

        <div class="po-section">
            <div class="po-title">Taxes and Payment</div>
            <div style="display: flex; justify-content: space-between;">
                <div>
                    <strong>Applicable Taxes:</strong>
                    <ul>
                        {''.join([f"<li>{tax}: {rate}</li>" for tax, rate in po_data['Applicable Taxes'].items()])}
                    </ul>
                </div>
                <div>
                    <p><strong>Payment Terms:</strong> {po_data['Terms of Payment']}</p>
                </div>
            </div>
        </div>

        <div class="po-section">
            <div class="po-title">Remarks and Instructions</div>
            <ul>
                {''.join([f"<li>{remark}</li>" for remark in po_data['Other Remarks/Instructions']])}
            </ul>
        </div>

        <div class="po-section">
            <div class="po-title">Customer Contact</div>
            <p><strong>Phone:</strong> {po_data['Customer Contact']['Phone']}</p>
            <p><strong>Email:</strong> {po_data['Customer Contact']['Email']}</p>
            <p><strong>GST Number:</strong> {po_data['Customer Contact']['GST Number']}</p>
        </div>
    </div>
    """

    # Display the HTML
    display(HTML(html_content))

# Example usage
def display_purchase_order(po_json_str):
    """
    Parse JSON string and display purchase order

    Args:
        po_json_str (str): JSON string of purchase order
    """
    try:
        po_data = json.loads(po_json_str)
        format_purchase_order(po_data)
    except json.JSONDecodeError:
        print("Invalid JSON format. Please provide a valid JSON string.")


In [ ]:
display_purchase_order(ab)

Item Name,Quantity,Rate per Unit,Unit,Delivery Schedule
Travalent screw washer,"300,000",₹0.98,Nos,"04/12: 60000 units, 08/12: 60000 units, 12/12: 60000 units, 18/12: 60000 units, 24/12: 60000 units"
SCREW WASHER ASSEMBLY MO-0,"160,000",₹0.97,Nos,"04/12: 20000 units, 10/12: 20000 units, 20/12: 40000 units, 25/12: 40000 units, 30/12: 40000 units"
